In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification   ## huggingface 
import torch   ## Pytorch
import torch.nn.functional as F         ## to convert logit to probability score.

In [2]:
!where python

C:\Users\efrui\DataspellProjects\Sentiment\myenv\Scripts\python.exe
C:\Program Files\Python313\python.exe
C:\Users\efrui\anaconda3\python.exe
C:\Users\efrui\AppData\Local\Microsoft\WindowsApps\python.exe


## Read data

In [16]:
import os
print(os.getcwd())


C:\Users\efrui\DataspellProjects\Sentiment\notebook


In [4]:
df = pd.read_csv("../data/cleaned/clean_loan_data.csv")



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Text               24000 non-null  object 
 1   Income             24000 non-null  int64  
 2   Credit_Score       24000 non-null  int64  
 3   Loan_Amount        24000 non-null  int64  
 4   DTI_Ratio          24000 non-null  float64
 5   Employment_Status  24000 non-null  object 
 6   Approval           24000 non-null  object 
 7   text_length        24000 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 1.5+ MB


## 1. Sentiment Analysis



For this sentiment analysis, i will be using RoBERTa from facebook ai.
it is a variant of BERT (Bidirectional Encoder Representations from Transformers)


It's trained bidirectionally, meaning it understands the meaning of words based on all their context (both left and right).MAde to understand tweeter english language, the sentiment behind each text.


classifies each text as either Negative, Neutral, or positive.

And also a better performer than BERT on many NLP tasks.

### Load Pretrained Tokenizer and Model using the Roberta base sentiment for text classification. 

In [7]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

C:\Users\efrui\DataspellProjects\Sentiment\myenv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\efrui\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Error while downloading from https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/pytorch_model.bin: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


pytorch_model.bin:  88%|########8 | 440M/499M [00:00<?, ?B/s]

In [8]:
# create label mapping for negative,neutral and positive as 0,1,2
labels = ['negative', 'neutral', 'positive']


### Logit prediction function

In [9]:
def predict_sentiment(text):
    """ 
    Predicts sentiment for a text
    text(str) : the input statement undergoing sentiment analysis.
    returns: sentiment(str) : the predicted sentiment of the input statement.(positive, negative, neutral)
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True) 
    # Converts to tensors (PyTorch format), trunctates to fit models input limits.
    with torch.no_grad(): #Disables gradient computation 
        # We’re only running inference (no training)
        logits = model(**inputs).logits # raw model output
    probs = F.softmax(logits, dim=1) # converts logits to probabilities
    sentiment = labels[probs.argmax().item()]  # Pick the label with highest probability
    return sentiment

#### Application

Test Application

In [34]:
text = 'I want to open a store for all things girly cosmetics'

In [14]:
sentiment = predict_sentiment(text)

In [15]:
sentiment

'positive'

Process Text column in batches:

In [19]:
def process_in_batches(df, batch_size=10, checkpoint_file='../data/cleaned/sentiment_checkpoint.csv'):
    """
    Processes sentiment predictions in batches and saves progress in ( checkpointing).
    Args:
        df : The DataFrame containing the data (`Text` column).
        batch_size (int): Number of rows to process per batch.
        checkpoint_file (str): Path to the checkpoint file (CSV) where each progress will be saved.
    Returns:
            None
        """
    # Check if there is a checkpoint file and load it
    try:
        completed = pd.read_csv(checkpoint_file)
        completed_indices = set(completed.index)
        print(f"Loaded checkpoint. {len(completed_indices)} rows already processed.")
    except FileNotFoundError:
        completed = df.copy()
        completed['Sentiment'] = None
        completed_indices = set()
        print("No checkpoint found. Starting from scratch.")
        
    batch_results =[] # temp list to store batch results.
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = start_idx + batch_size
        batch = df.iloc[start_idx:end_idx]
    
        # Skip rows that have already been processed
        batch = batch[~batch.index.isin(completed_indices)]
    
        if batch.empty:
            continue
    
        # Process each row in the batch
        for idx, row in batch.iterrows():
            sentiment = predict_sentiment(row['Text']) # predicts sentiment
            
            result = {
                'index': idx, # index of the row
                **row.to_dict(), # add every other columns from the row
                'Sentiment': sentiment # add sentiment
            }
            batch_results.append(result)
        #convert batch result to df
        batch_df = pd.DataFrame(batch_results).set_index('index')
        
        #update completed df.
        completed.update(batch_df)
        
        # Save the checkpoint after processing each batch
        completed.to_csv(checkpoint_file, index=False)
        print(f"Processed up to index {end_idx}. Checkpoint saved.")

    print("Processing complete.")

Call batch function

In [ ]:
process_in_batches(df, batch_size=1000, checkpoint_file='../data/cleaned/sentiment_checkpoint.csv')


No checkpoint found. Starting from scratch.
Processed up to index 1000. Checkpoint saved.
Processed up to index 2000. Checkpoint saved.
Processed up to index 3000. Checkpoint saved.
Processed up to index 4000. Checkpoint saved.
Processed up to index 5000. Checkpoint saved.
Processed up to index 6000. Checkpoint saved.
Processed up to index 7000. Checkpoint saved.
Processed up to index 8000. Checkpoint saved.
Processed up to index 9000. Checkpoint saved.
Processed up to index 10000. Checkpoint saved.
Processed up to index 11000. Checkpoint saved.
Processed up to index 12000. Checkpoint saved.
Processed up to index 13000. Checkpoint saved.
Processed up to index 14000. Checkpoint saved.
Processed up to index 15000. Checkpoint saved.
Processed up to index 16000. Checkpoint saved.
Processed up to index 17000. Checkpoint saved.
Processed up to index 18000. Checkpoint saved.
Processed up to index 19000. Checkpoint saved.
Processed up to index 20000. Checkpoint saved.
Processed up to index 210

Import new data 

In [25]:
df = pd.read_csv('../data/cleaned/sentiment_checkpoint.csv')
df[['Text', 'Sentiment']].head(17)


,Text,Sentiment
0,I need a loan to pay for an international vaca...,neutral
1,I want to make home improvements like installi...,positive
2,"I need a loan for home renovation, including a...",neutral
3,I need funds to buy new furniture and applianc...,neutral
4,I need a loan to start a small business.,neutral
5,I need a loan to repair my car after an accident.,negative
6,I need financial help to cover maternity and d...,neutral
7,I need money to open a small bookstore and café.,neutral
8,I need a loan to install a security system at ...,neutral
9,I need a loan to cover emergency expenses afte...,neutral


## Topic Modeling


Topic modeling is an unsupervised NLP technique used to automatically discover hidden themes or topics in a collection of documents

By applying topic modeling to these justifications, we can:

        1. Group similar justifications: 	Reveals themes like business expansion, medical bills, Vacation etc.
        2. Understand applicant needs:  Segments borrowers by reason without labeling
        3. Improve decision making	: create risk scoring by reason
        4. Feed into other models:	Use topics as features in my approval agent.

In [26]:
from bertopic import BERTopic

Why BERTopic:

        1. Uses transformer embeddings (more context aware, better understanding of text)
        2. Clusters similar ideas even if the wording is different
        3. Produces interpretable results (eg Top 10 keywords per topic)
        4. Works well on short text, which is perfect for loan justifications with medium length of 62 characters

In [27]:
from sentence_transformers import SentenceTransformer

In [28]:
# Reset index
df = df.reset_index(drop=True)

In [29]:
df.head()

,Text,Income,Credit_Score,Loan_Amount,DTI_Ratio,Employment_Status,Approval,text_length,sentiment,Sentiment
0,I need a loan to pay for an international vaca...,26556,581,8314,79.26,employed,Rejected,66,neutral,neutral
1,I want to make home improvements like installi...,197392,389,111604,22.14,employed,Rejected,62,positive,positive
2,"I need a loan for home renovation, including a...",44561,523,34118,45.44,employed,Rejected,63,neutral,neutral
3,I need funds to buy new furniture and applianc...,190363,729,118757,10.22,unemployed,Rejected,62,neutral,neutral
4,I need a loan to start a small business.,61853,732,19210,44.13,employed,Approved,40,neutral,neutral


#### Preprocessing

In [88]:
import re
import nltk

# Make sure required NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\efrui\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\efrui\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\efrui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\efrui\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [89]:
df.drop(columns=['sentiment'])

,Text,Income,Credit_Score,Loan_Amount,DTI_Ratio,Employment_Status,Approval,text_length,Sentiment
0,i need a loan to pay for an international vaca...,26556,581,8314,79.26,employed,Rejected,66,neutral
1,i want to make home improvements like installi...,197392,389,111604,22.14,employed,Rejected,62,positive
2,i need a loan for home renovation including a ...,44561,523,34118,45.44,employed,Rejected,63,neutral
3,i need funds to buy new furniture and applianc...,190363,729,118757,10.22,unemployed,Rejected,62,neutral
4,i need a loan to start a small business,61853,732,19210,44.13,employed,Approved,40,neutral
...,...,...,...,...,...,...,...,...,...
23995,i need funds to pay for my daughters college t...,195242,817,16403,24.32,employed,Approved,54,neutral
23996,i need financial assistance to launch my own y...,150246,729,101572,9.97,employed,Rejected,79,neutral
23997,i need money to open a small bookstore and caf,64571,650,30533,57.35,employed,Rejected,48,neutral
23998,i want to buy a car for my rideshare business ...,115825,418,89837,10.37,unemployed,Rejected,54,positive


In [90]:
#strip text to lower case
df['Text'] = df['Text'].str.lower()

In [91]:
# KEep only alphabets and space , no punctuations
df['Text'] = df['Text'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

In [93]:
# Next, tokenize split sentences into individual words
df['tokens'] = df['Text'].apply(word_tokenize)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\efrui/nltk_data'
    - 'C:\\Users\\efrui\\DataspellProjects\\Sentiment\\myenv\\nltk_data'
    - 'C:\\Users\\efrui\\DataspellProjects\\Sentiment\\myenv\\share\\nltk_data'
    - 'C:\\Users\\efrui\\DataspellProjects\\Sentiment\\myenv\\lib\\nltk_data'
    - 'C:\\Users\\efrui\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


### Initialize the embedding model

We use a SentenceTransformer model to convert the text into dense numerical vectors that capture meaning and context.


It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search [https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2]

In [32]:
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') # Small, fast, high quality


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Test application 

In [74]:
# generating text list demo from the df[Text] column.
text_list = df['Text'].head(2000).astype(str).tolist()

perform Vector embedding

In [75]:
embedding =embedding_model.encode(text_list)
print(embedding)

[[ 0.02358866 -0.02246133 -0.02415422 ... -0.05399248 -0.0308361
  -0.09526329]
 [ 0.00551293  0.01618007  0.01452278 ...  0.03577202 -0.0062503
   0.02331941]
 [ 0.02921814 -0.06256145 -0.00040705 ... -0.01655832 -0.02784156
  -0.08535296]
 ...
 [ 0.05590715 -0.05136776 -0.0378224  ... -0.0624883  -0.00517669
  -0.07733417]
 [ 0.03967594 -0.04740962 -0.031529   ... -0.09508014  0.01880769
  -0.06381962]
 [ 0.02255474 -0.05363455  0.02863037 ...  0.01889588 -0.03059375
  -0.07006724]]


In [76]:
#Run Berttopic for topic clustering and modeling. 

topic_model = BERTopic(embedding_model=embedding_model, language="english")



In [77]:
# topics: topic number assigned to each loan justification
# probs: confidence score for the topic assignment

topics, probs = topic_model.fit_transform(text_list)

In [78]:
# Print the output together with the input text
for text, topic, prob in zip(text_list, topics, probs):
    print(f"Text: {text}")
    print(f"Topic: {topic}, Probability: {prob}")
    print("-" * 50)  # seperator

Text: I need a loan to pay for an international vacation with my family.
Topic: 43, Probability: 0.9035604105112433
--------------------------------------------------
Text: I want to make home improvements like installing solar panels.
Topic: 53, Probability: 0.08432323418221335
--------------------------------------------------
Text: I need a loan for home renovation, including a kitchen remodel.
Topic: 68, Probability: 0.3851426945915192
--------------------------------------------------
Text: I need funds to buy new furniture and appliances for my house.
Topic: 67, Probability: 1.0
--------------------------------------------------
Text: I need a loan to start a small business.
Topic: 57, Probability: 0.5939444995371724
--------------------------------------------------
Text: I need a loan to repair my car after an accident.
Topic: 51, Probability: 0.302197434434709
--------------------------------------------------
Text: I need financial help to cover maternity and delivery expense

In [79]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,42,0_specialized_shop_auto_repair,"[specialized, shop, auto, repair, tools, purch...",[I need money to purchase specialized tools fo...
1,1,41,1_develop_app_startup_mobile,"[develop, app, startup, mobile, launch, and, l...",[I need a loan to develop and launch my mobile...
2,2,37,2_truck_food_own_purchase,"[truck, food, own, purchase, start, business, ...",[I need money to purchase a food truck and sta...
3,3,37,3_swimming_pool_backyard_install,"[swimming, pool, backyard, install, help, in, ...",[I need financial help to install a swimming p...
4,4,36,4_jewelry_brand_online_handmade,"[jewelry, brand, online, handmade, launch, fun...",[I need funds to launch my handmade jewelry br...
...,...,...,...,...,...
66,66,19,66_based_catering_home_start,"[based, catering, home, start, business, money...",[I need money to start a home-based catering b...
67,67,18,67_house_furniture_appliances_new,"[house, furniture, appliances, new, buy, funds...",[I need funds to buy new furniture and applian...
68,68,18,68_remodel_renovation_kitchen_including,"[remodel, renovation, kitchen, including, home...","[I need a loan for home renovation, including ..."
69,69,17,69_house_furniture_appliances_new,"[house, furniture, appliances, new, buy, funds...",[I need funds to buy new furniture and applian...


Insight from test:

The mdel performance is not peak, It doesnt cluster well on short text, as we see it has some unneccasry clusters. 72 clusters for 2000 records is a bit too over the place. I also see some non related representation groups.



Lets try other embedding models.

2nd Embedding Model:
 

In [70]:
embedding_model2 = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [71]:
topic_model2 = BERTopic(embedding_model=embedding_model2, language="english")

Test on same test_list

In [80]:
topics2, probs2 = topic_model2.fit_transform(text_list)

In [81]:
topic_model2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,42,0_specialized_shop_auto_repair,"[specialized, shop, auto, repair, tools, purch...",[I need money to purchase specialized tools fo...
1,1,41,1_develop_app_startup_mobile,"[develop, app, startup, mobile, launch, and, l...",[I need a loan to develop and launch my mobile...
2,2,37,2_truck_food_own_purchase,"[truck, food, own, purchase, start, business, ...",[I need money to purchase a food truck and sta...
3,3,37,3_swimming_pool_backyard_install,"[swimming, pool, backyard, install, help, in, ...",[I need financial help to install a swimming p...
4,4,36,4_jewelry_brand_online_handmade,"[jewelry, brand, online, handmade, launch, fun...",[I need funds to launch my handmade jewelry br...
...,...,...,...,...,...
65,65,22,65_written_self_publish_book,"[written, self, publish, book, market, money, ...",[I need money to publish and market my self-wr...
66,66,21,66_tech_solutions_focused_ai,"[tech, solutions, focused, ai, startup, own, o...",[I need money to launch my own tech startup fo...
67,67,19,67_based_catering_home_start,"[based, catering, home, start, business, money...",[I need money to start a home-based catering b...
68,68,18,68_remodel_renovation_kitchen_including,"[remodel, renovation, kitchen, including, home...","[I need a loan for home renovation, including ..."


#### Parameter tuning

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import umap
import hdbscan

In [62]:

#  Vectorizer for bag-of-words
vectorizer_model = CountVectorizer(stop_words="english", min_df=10)


In [63]:

#  UMAP for dimensionality reduction
umap_model = umap.UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')


In [64]:

#  HDBSCAN for clustering
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


In [65]:

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [68]:

# Combine into BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,         
    vectorizer_model=vectorizer_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    ctfidf_model=ctfidf_model,
    calculate_probabilities=True,
    verbose=True
)

#### Second test

In [69]:
# topics: topic number assigned to each loan justification
# probs: confidence score for the topic assignment

topics, probs = topic_model.fit_transform(text_list)
# Print the output together with the input text
for text, topic, prob in zip(text_list, topics, probs):
    print(f"Text: {text}")
    print(f"Topic: {topic}, Probability: {prob}")
    print("-" * 50)  # readable seperator

2025-04-15 05:54:00,910 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2025-04-15 05:54:03,368 - BERTopic - Embedding - Completed ✓
2025-04-15 05:54:03,369 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-15 05:54:05,131 - BERTopic - Dimensionality - Completed ✓
2025-04-15 05:54:05,132 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-15 05:54:05,234 - BERTopic - Cluster - Completed ✓
2025-04-15 05:54:05,237 - BERTopic - Representation - Extracting topics from clusters using representation models.


TypeError: object of type 'numpy.float64' has no len()